# Project 1

In [2]:
import pandas as pd 
import numpy as np
import seaborn as sns
from numpy import linalg as la
from io import StringIO
from tabulate import tabulate
from matplotlib import pyplot as plt
from scipy.stats import chi2
import LinearModelsWeek2_ante as lm
import statsmodels.api as sm
import LinearModelsWeek2_post as lm1


In [3]:
# Load data
dat = pd.read_csv('firms.csv')
desired_years = [1968, 1969, 1970]
mask = dat['year'].isin(desired_years)

# Use the mask to filter the DataFrame
filtered_df = dat[mask]
filtered_df.year.unique()
#print(filtered_df)



array([1968, 1969, 1970], dtype=int64)

In [4]:
def demeaning_matrix(T):
    Q_T = np.identity(T) - (1/T) * np.ones((T,T))
    return Q_T

T = 3
Q_T = demeaning_matrix(T)
print(f'Demeaning matrix for T={T} \n', Q_T)

Demeaning matrix for T=3 
 [[ 0.66666667 -0.33333333 -0.33333333]
 [-0.33333333  0.66666667 -0.33333333]
 [-0.33333333 -0.33333333  0.66666667]]


In [5]:
y = filtered_df.ldsa.values
xi = []
ID = 0


for i in range(len(filtered_df)):
        if i%3 == 0: 
            yr = 1
        elif i%3 == 2:
            yr = 3
        else: 
            yr = 2
        if i%3 == 0: 
            ID = ID+1
       
        row = [ID, yr, filtered_df.lemp.values[i], filtered_df.lcap.values[i]]
        xi.append(row)
xi = np.array(xi)

#print("xi:")
#print(xi.shape)
#print("y:")#
yi = y.reshape(-1,1)
#print(yi.shape)



#print(y.T)

In [6]:
y_demean = lm.perm(Q_T,yi,t=T)
x_demean = lm.perm(Q_T,xi,t=T)
x_demean1 = x_demean[:,2:4]

#print (x_demean)
#print(x_demean)
#print(y_demean)

In [7]:
def check_rank(x):
    print(f'Rank of demeaned x: {la.matrix_rank(x)}')
    lambdas, V = la.eig(x.T@x)
    np.set_printoptions(suppress=True)  # This is just to print nicely.
    print(f'Eigenvalues of within-transformed x: {lambdas.round(decimals=0)}')

# Check rank of demeaned x
check_rank(x_demean)

Rank of demeaned x: 3
Eigenvalues of within-transformed x: [  0. 882.   9.   5.]


In [8]:
#pd.DataFrame(x_demean)

In [9]:
x_demean_vars = (pd.DataFrame(x_demean[:,2:])).to_numpy()


label_x_fe = ['lemp', 'lcap']
N = 441
fe_result = lm.estimate(y_demean, x_demean_vars, N=N, T=T)
label_y = "ldsa"
# Print results
a = lm.print_table((label_y, label_x_fe), fe_result, title='FE regression', floatfmt='.4f')
b = pd.DataFrame(a)

print(b.style.to_latex())

FE regression
Dependent variable: ldsa

        Beta      Se    t-values
----  ------  ------  ----------
lemp  0.6004  0.0282     21.2592
lcap  0.0502  0.0312      1.6103
R² = 0.284
σ² = 0.005


\begin{tabular}{l}
\end{tabular}



In [10]:
null_hypothesis = "Coefficients of 'lemp' and 'lcap' are jointly equal to 1."
beta_hat = fe_result['b_hat']
beta_hat = beta_hat.reshape(-1, 1)
num_restrictions = 2
V_hat = fe_result['cov']
wald_statistic = float((beta_hat - 1).T @ np.linalg.inv(V_hat) @ (beta_hat - 1))
alpha = 0.05  # Significance level
critical_value = chi2.ppf(1 - alpha, df=num_restrictions)
p_value = 1 - chi2.cdf(wald_statistic, df=num_restrictions)
if wald_statistic > critical_value:
    result = "Reject the null hypothesis"
else:
    result = "Fail to reject the null hypothesis"

# Print the results.
print("Null Hypothesis:", null_hypothesis)
print("Wald Statistic:", wald_statistic)
print("Critical Value:", critical_value)
print("P-Value:", p_value)
print("Result:", result)





Null Hypothesis: Coefficients of 'lemp' and 'lcap' are jointly equal to 1.
Wald Statistic: 1535.636574184482
Critical Value: 5.991464547107979
P-Value: 0.0
Result: Reject the null hypothesis


In [33]:


X = sm.add_constant(x_demean_vars)
model = sm.OLS(y, X).fit()
residuals = model.resid
test_result = sm.stats.diagnostic.het_breuschpagan(residuals, X)
p_value = test_result[1]
alpha = 0.05
print(p_value)
if p_value < alpha:
    print("Reject the null hypothesis: Error term is not strictly exogenous.")
else:
    print("Fail to reject the null hypothesis: Error term is strictly exogenous.")

0.9879283282975101
Fail to reject the null hypothesis: Error term is strictly exogenous.


In [20]:
def serial_corr(y, x, T):
    b_hat = lm1.est_ols(y, x)
    e = y - x@b_hat
    L_T = np.eye(T, k=-1)
    L_T = L_T[1:]
    e_l = lm.perm(L_T, e)
    I_T = np.eye(T, k=0)
    I_T = I_T[1:]
    e = lm.perm(I_T, e)
    return lm.estimate(e, e_l,N=N,T=T-1)



In [21]:
corr_result = serial_corr(y_demean, x_demean, T)

# Print results
label_ye = 'OLS residual, e\u1d62\u209c'
label_e = ['e\u1d62\u209c\u208B\u2081']
lm.print_table(
    (label_ye, label_e), corr_result, 
    title='Serial Correlation', floatfmt='.4f'
)


Serial Correlation
Dependent variable: OLS residual, eᵢₜ

          Beta      Se    t-values
-----  -------  ------  ----------
eᵢₜ₋₁  -0.3638  0.0318    -11.4285
R² = 0.129
σ² = 0.004


In [31]:
Xnull_hypothesis = "Coefficients of 'lemp' and 'lcap' are jointly equal to 1."
u_hat = y_demean-x_demean_vars@beta_hat
sandwich_brød = (np.linalg.inv(x_demean_vars.T@x_demean_vars))
sandwich_kød = (np.sum(x_demean_vars.T@u_hat@u_hat.T@x_demean_vars))
var_robust = sandwich_brød*sandwich_kød*sandwich_brød
R_vec = np.array([1,1]).reshape(1,-1)   
r = 1
førsteled = R_vec@beta_hat-r
andetled = np.linalg.inv(R_vec@var_robust@R_vec.T)
wald_statistic = float(førsteled@andetled@førsteled.T)
#print(beta_hat)
##print('{:.4f}'.format(wald_statistic))
alpha = 0.05  # Significance level
critical_value = chi2.ppf(1 - alpha, df=num_restrictions)
#print(critical_value)
p_value = 1 - chi2.cdf(wald_statistic, df=num_restrictions)
print(f'With a critical value of {critical_value:.4f}, and a Wald statistic of {round(wald_statistic)} \nWe have a p-value of {p_value:.4f}')


With a critical value of 5.9915, and a Wald statistic of 256345089501154634735688876032 
We have a p-value of 0.0000
